In [1]:
from BiblioMeter_GUI.Page_Classes import App_Test

app = App_Test()
app.mainloop()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AC265100\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AC265100\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AC265100\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
